In [1]:
import sys
from astropy.io import fits
import glob
import numpy as np
#import npat
import time
import resource
import subprocess
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

In [2]:
def plotImage(image_data):
    plt.figure()
    plt.imshow(image_data)
    plt.colorbar()
    plt.title('Imagen FITS')
    plt.xlabel('Pixel X')
    plt.ylabel('Pixel Y')
    plt.savefig('Image-disk.png')
    plt.show()

def plotVelocity(X, Y, vx, vy):
    
    # Visualización de las componentes de velocidad
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.pcolormesh(X, Y, vx)
    plt.colorbar(label='Velocidad en X')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Componente de Velocidad en X')
    
    plt.subplot(1, 2, 2)
    plt.pcolormesh(X, Y, vy)
    plt.colorbar(label='Velocidad en Y')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Componente de Velocidad en Y')
    
    plt.tight_layout()
    plt.savefig('Velocity-disk.png')
    plt.show()

In [3]:
def plotVelocityField(X, Y, Vx, Vy, npix):
    
    speed = np.sqrt(Vx**2 + Vy**2)
    lw = 5*speed / speed.max()

    plt.figure(figsize=(10, 10))
    plt.streamplot(X, Y, Vx, Vy, density=0.6, color='k', linewidth=lw)
    plt.colorbar(label='Velocity magnitude')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Streamplot of Velocity Field')
    plt.show()

In [4]:
pnt = False

In [5]:
# Obtener campos de velocidad e interpolar datos para cambiar dimension de los arreglos ---> Actualmente se trabaja con dust1v
def get_velocity(file_path_field, out, npix):

    # Leer los datos del archivo 'domain_y.dat', excluyendo los primeros 3 y los últimos 4 valores
    # En su interior contiene los radios (r) --> coordenadas radiales 
    r = np.loadtxt(file_path_field + 'domain_y.dat')[3:-4]
    
    # Leer los datos del archivo 'domain_x.dat', excluyendo el último valor
    # En su interior contiene los ángulos (φ) --> coordenadas angulares (azimutales)
    phi = np.loadtxt(file_path_field + 'domain_x.dat')[:-1]
    
    if pnt: 
        print("R shape: ", r.shape)
        print("Phi shape: ", phi.shape)
    
    # Determinar la cantidad de elementos en r y phi
    nr = len(r)
    nphi = len(phi)
    
    # Crear una malla de coordenadas polares (Phi, R) usando phi y r
    Phi, R = np.meshgrid(phi, r)
    
    # Convertir las coordenadas polares (Phi, R) a coordenadas cartesianas (X, Y)
    X = R * np.cos(Phi)
    Y = R * np.sin(Phi)
    
    if pnt:
        print("Coordenadas cartesianas - shape: ", X.shape, Y.shape)
        print("Original X range:", X.min(), X.max())
        print("Original Y range:", Y.min(), Y.max())
    
    # Velocidades en las direcciones radial en coordenadas polares
    # Cada fila -> posición radial; Cada columna -> posición azimutal
    vy = np.fromfile(file_path_field + 'dust1vy{:d}.dat'.format(out), dtype=np.float64).reshape(nr, nphi)
    
    # Velocidades en las direcciones azimutal en coordenadas polares
    # Cada fila -> posición radial; Cada columna -> posición azimutal
    vx = np.fromfile(file_path_field + 'dust1vx{:d}.dat'.format(out), dtype=np.float64).reshape(nr, nphi)
    
    if pnt: 
        print("Vy shape: ", vy.shape)
        print("Vx shape: ", vx.shape)
        print("Vx Max: ", vx.max(), " Vx Min: ", vx.min())
        print("Vy Max: ", vy.max(), " Vy Min: ", vy.min())
        plotVelocity(X, Y, vx, vy) 
        #plotVelocityField(X, Y, vx, vy, npix) 

    # Crear una malla de coordenadas polares para la nueva grilla
    new_r = np.linspace(R.min(), R.max(), npix)
    new_phi = np.linspace(Phi.min(), Phi.max(), npix)
    new_Phi, new_R = np.meshgrid(new_phi, new_r)

    if pnt:
        print("Grilla : ", new_Phi.shape, new_R.shape)
        print("Nuevas coordenadas polares - shape: ", new_Phi.shape, new_R.shape)
        print("New Phi range:", new_Phi.min(), new_Phi.max())
        print("New R range:", new_R.min(), new_R.max())

    # Convertir las nuevas coordenadas polares a coordenadas cartesianas para la interpolación
    new_X = new_R * np.cos(new_Phi)
    new_Y = new_R * np.sin(new_Phi)
    
    if pnt:
        print("Grilla : ", new_X.shape, new_Y.shape)
        print("Nuevas coordenadas cartesianas - shape: ", new_X.shape, new_Y.shape)
        print("New X range:", new_X.min(), new_X.max())
        print("New Y range:", new_Y.min(), new_Y.max())
        
    # Interpolar los datos de velocidad a la nueva grilla - nearest es más rapido y no tienen problemas con valores atipicos
    new_vx = griddata((X.ravel(), Y.ravel()), vx.ravel(), (new_X, new_Y), method='nearest')
    new_vy = griddata((X.ravel(), Y.ravel()), vy.ravel(), (new_X, new_Y), method='nearest')
    
    
    if pnt:
        print("Nuevas dimensiones c.p vx, vy - shape: ", new_vx.shape, new_vy.shape)
        print("new_Vx Max: ", new_vx.max(), " new_Vx Min: ", new_vx.min())
        print("new_Vy Max: ", new_vy.max(), " new_Vy Min: ", new_vy.min())
        print("Number of NaNs in new_vx:", np.isnan(new_vx).sum())
        print("Number of NaNs in new_vy:", np.isnan(new_vy).sum())

    if pnt:
        #plotVelocity(new_X, new_Y, new_vx, new_vy)
        plotVelocity(X, Y, vx, vy)
        #plotVelocityField(new_X, new_Y, new_vx, new_vy, npix)

    return new_vx, new_vy
        

In [6]:
#__________________________________GUARDAR DATOS___________________________________

##GENERAL SETTING####
#simulation = 135
simulation = 199
offset = 0
time_step = 20
npix = 300

# Archivos con errores en las simulaciones
err_sim = [9, 18, 26, 29, 36, 71, 83, 91, 92, 93, 134, 136, 143, 151, 153, 170, 182, 196, 197]

##VELOCITY SETTING###
images_data_norm_tot = []
output_np_file_name_img = "./numpyFile_simulation_images"

##VELOCITY SETTING###
v_time = 19 # Indice del time step para obtener el campo de velocidades (19 last)
vfield_data_norm_x_tot = []
vfield_data_norm_y_tot = []
output_np_file_name_vel_x = "./numpyFile_simulation_velocity_x"
output_np_file_name_vel_y = "./numpyFile_simulation_velocity_y"

In [7]:
for sim in range(simulation):

    if (sim+offset) in err_sim:
        continue

    images_data_norm_by_sim = []

    for t in range(1, time_step):
    
        ###TIME####
        # Ruta donde esten guardadas los imagenes
        file_path_time = f'/disk2/alma/pipeline/RT/All_sim_pix{npix:01d}/sim_{(sim+offset):01d}/output_{t:01d}/image_out_wl1300.fits'

        hdulist = fits.open(file_path_time, mode='readonly', do_not_scale_image_data=True)
        image_data = hdulist[0].data
        hdulist.close()
        image_data = np.squeeze(image_data) #arreglar dimension

        # Normalizar datos antes de guardar
        aux_norm = image_data - image_data.min()
        image_data_norm = np.divide(aux_norm, aux_norm.max(), dtype=np.float32)
        images_data_norm_by_sim.append(image_data_norm)

        if(t == (time_step-1) and pnt): 
            plotImage(image_data)
            plotImage(image_data_norm)

    images_data_norm_tot.append(images_data_norm_by_sim)
    ###VELOCITY###
    # Ruta donde estan guardadas las velocidades
    file_path_field = f'/disk2/alma/pipeline/hydro/outputs/Alma_outputs/sim_{(sim+offset):01d}/'

    # Obtener velocidades del polvo (dust1) del ultimo paso de tiempo (cambiar función a gusto)
    vfield_data_x, vfield_data_y = get_velocity(file_path_field, v_time, npix)
    
    # Normalizar datos antes de guardar
    aux_norm = vfield_data_x - vfield_data_x.min()
    vfield_data_norm_x = np.divide(aux_norm, aux_norm.max(), dtype=np.float32)
    vfield_data_norm_x_tot.append(vfield_data_norm_x)

    aux_norm = vfield_data_y - vfield_data_y.min()
    vfield_data_norm_y = np.divide(aux_norm, aux_norm.max(), dtype=np.float32)
    vfield_data_norm_y_tot.append(vfield_data_norm_y)

images_data_norm_tot_np = np.asarray(images_data_norm_tot)
vfield_data_norm_x_tot_np = np.asarray(vfield_data_norm_x_tot)
vfield_data_norm_y_tot_np = np.asarray(vfield_data_norm_y_tot)

print("===== SAVING numpy files to local disk")
print(images_data_norm_tot_np.shape)
print(vfield_data_norm_x_tot_np.shape)
print(vfield_data_norm_y_tot_np.shape)

np.save(output_np_file_name_img, images_data_norm_tot_np)
np.save(output_np_file_name_vel_x, vfield_data_norm_x_tot_np)
np.save(output_np_file_name_vel_y, vfield_data_norm_y_tot_np)

===== SAVING numpy files to local disk
(180, 19, 300, 300)
(180, 300, 300)
(180, 300, 300)
